# SQL in Python - Connecting to and retrieving data from PostgreSQL

Previously, you have learned how to connect to a SQL database by using a SQL client such as DBeaver. Apart from connecting to databases, DBeaver also allows you to run SQL queries against the database, create new tables and populate them with data as well as retrieving the data.

Python also allows executing SQL queries and getting the result into a Python object, for example a Pandas data frame. Instead of exporting a .csv file from DBeaver you can directly get the data you need into Python and continue your work. In addition we can reduce the steps by connecting to the database from Python directly, eliminating the need for a separate SQL client.

After you have the data in Python in the required shape you can export the data into a .csv file. This file is for your own reference, please avoid sending .csv files around - database is the point of reference when it comes to data. 

Having a copy of a .csv file (or another format) can speed up your analysis work. Imagine that the query takes 25 minutes to run. If you made some mistakes in your Python code you might need to go back to the original dataset. Instead of having to rerun the SQL query and having to wait you can read in the .csv file you have previously saved on your hard disk into Python and continue with your analysis work. 

**In this notebook you will see 2 ways to connect to SQL-Databases and export the data to a CSV file**


## Creating a connection to a PostgreSQL database with Python

There are 2 python packages that are the "go-to" when it comes to connecting to SQL-Databases: `psycopg2` and `sqlalchemy` 

### Connecting via psycopg2

In [3]:
import pandas as pd
import psycopg2


In order to create a connection to our PostgreSQL database we need the following information:

- host = the address of the machine the database is hosted on
- port = the virtual gate number through which communication will be allowed
- database = the name of the database
- user = the name of the user
- password = the password of the user

Because we don't want that the database information is published on github we put it into a `.env` file which is added into the `.gitignore`. 
In these kind of files you can store information that is not supposed to be published.
With the `dotenv` package you can read the `.env` files and get the variables.
(We will share the file with you on Slack!)


In [40]:
import os
from dotenv import load_dotenv

# READ database user password host and port from .env
load_dotenv()

DATABASE = os.postgres('DATABASE')
USER_DB = os.ktnipclustering('USER_DB')
PASSWORD = os.mialovesicecream('PASSWORD')
HOST = os.ds-sql-playground.c8g8r1deus2v.eu-central-1.rds.amazonaws.com('HOST')
PORT = os.5432('PORT')

SyntaxError: invalid decimal literal (1735037317.py, line 10)

The function from the psycopg2 package to create a connection is called `connect()`.
`connect()` expects the parameters listed above as input in order to connect to the database.

In [3]:
# Create connection object conn
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

### Retrieving data from the database with psycopg2

Before we can use our connection to get data, we have to create a cursor. A cursor allows Python code to execute PostgreSQL commmands in a database session.
A cursor has to be created with the `cursor()` method of our connection object conn.

In [4]:
cur = conn.cursor()

Now we can run SQL-Queries with `cur.execute('QUERY')` and then run `cur.fetchall()` to get the data:

In [5]:
cur.execute('SELECT * FROM eda.king_county_house_sales LIMIT 10')
cur.fetchall()

[(datetime.date(2014, 10, 13), 221900.0, 7129300520, 1),
 (datetime.date(2014, 12, 9), 538000.0, 6414100192, 2),
 (datetime.date(2015, 2, 25), 180000.0, 5631500400, 3),
 (datetime.date(2014, 12, 9), 604000.0, 2487200875, 4),
 (datetime.date(2015, 2, 18), 510000.0, 1954400510, 5),
 (datetime.date(2014, 5, 12), 1230000.0, 7237550310, 6),
 (datetime.date(2014, 6, 27), 257500.0, 1321400060, 7),
 (datetime.date(2015, 1, 15), 291850.0, 2008000270, 8),
 (datetime.date(2015, 4, 15), 229500.0, 2414600126, 9),
 (datetime.date(2015, 3, 12), 323000.0, 3793500160, 10)]

With `conn.close()` you can close the connection again.

In [6]:
#close the connection
conn.close()

But we want to work with the data. The easiest way is to import the data into pandas dataframes. We can use `pd.read_sql_query` or `pd.read_sql_table` or for convenience `pd.read_sql`.

This function is a convenience wrapper around read_sql_table and read_sql_query (for backward compatibility). It will delegate to the specific function depending on the provided input. A SQL query will be routed to read_sql_query , while a database table name will be routed to read_sql_table . Note that the delegated function might have more specific notes about their functionality not listed here.

In [7]:
# Open connection again because we closed it
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

In [8]:
# import the data into a pandas dataframe
query_string = "SELECT * FROM eda.king_county_house_sales LIMIT 10"
df_psycopg = pd.read_sql(query_string, conn)

In [9]:
#close the connection
conn.close()

In [10]:
df_psycopg.head()

,date,price,house_id,id
0,2014-10-13,221900.0,7129300520,1
1,2014-12-09,538000.0,6414100192,2
2,2015-02-25,180000.0,5631500400,3
3,2014-12-09,604000.0,2487200875,4
4,2015-02-18,510000.0,1954400510,5


In [17]:
#export the data to a csv-file
df_psycopg.to_csv('data/eda.csv',index=False)

### Connecting and retrieving data via SQLAlchemy

`sqlalchemy` works similarly. Here you have to create an engine with the database sting (a link that includes every information we entered in the conn object)

In [20]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

# READ database user password host and port from .env
load_dotenv()

DATABASE = os.getenv('DATABASE')
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

DB_STRING = os.getenv('DB_STRING')

#DB_STRING = f"postgresql://{USER_DB}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
db = create_engine(DB_STRING)



And then you can import that engine with a query into a pandas dataframe.

In [36]:
#import the data to a pandas dataframe
query_string = "SELECT * FROM eda.king_county_house_sales"
df_sqlalchemy = pd.read_sql(query_string, db)

In [37]:
df_sqlalchemy.head()

,date,price,house_id,id
0,2014-10-13,221900.0,7129300520,1
1,2014-12-09,538000.0,6414100192,2
2,2015-02-25,180000.0,5631500400,3
3,2014-12-09,604000.0,2487200875,4
4,2015-02-18,510000.0,1954400510,5


In [65]:
df_sqlalchemy.describe()

,price,house_id,id
count,2.159700e+04,2.159700e+04,21597.000000
mean,5.402966e+05,4.580474e+09,10799.000000
std,3.673681e+05,2.876736e+09,6234.661218
min,7.800000e+04,1.000102e+06,1.000000
25%,3.220000e+05,2.123049e+09,5400.000000
50%,4.500000e+05,3.904930e+09,10799.000000
75%,6.450000e+05,7.308900e+09,16198.000000
max,7.700000e+06,9.900000e+09,21597.000000


Because we don't want to run the queries over and over again we can export the data into a .csv file in order to use it in other notebooks as well. 

In [26]:
#export the data to a csv-file
df_sqlalchemy.to_csv('data/eda_house_sales.csv',index=False)

In [27]:
#import the data from a csv-file
df_import = pd.read_csv('data/eda_house_sales.csv')

In [60]:
df_import.head(10)

,date,price,house_id,id
0,2014-10-13,221900.0,7129300520,1
1,2014-12-09,538000.0,6414100192,2
2,2015-02-25,180000.0,5631500400,3
3,2014-12-09,604000.0,2487200875,4
4,2015-02-18,510000.0,1954400510,5
5,2014-05-12,1230000.0,7237550310,6
6,2014-06-27,257500.0,1321400060,7
7,2015-01-15,291850.0,2008000270,8
8,2015-04-15,229500.0,2414600126,9
9,2015-03-12,323000.0,3793500160,10


In [61]:
df_import.tail(10)

,date,price,house_id,id
21587,2014-08-25,507250.0,7852140040,21588
21588,2015-01-26,429000.0,9834201367,21589
21589,2014-10-14,610685.0,3448900210,21590
21590,2015-03-26,1010000.0,7936000429,21591
21591,2015-02-19,475000.0,2997800021,21592
21592,2014-05-21,360000.0,263000018,21593
21593,2015-02-23,400000.0,6600060120,21594
21594,2014-06-23,402101.0,1523300141,21595
21595,2015-01-16,400000.0,291310100,21596
21596,2014-10-15,325000.0,1523300157,21597


In [63]:
df_import.describe()

,price,house_id,id
count,2.159700e+04,2.159700e+04,21597.000000
mean,5.402966e+05,4.580474e+09,10799.000000
std,3.673681e+05,2.876736e+09,6234.661218
min,7.800000e+04,1.000102e+06,1.000000
25%,3.220000e+05,2.123049e+09,5400.000000
50%,4.500000e+05,3.904930e+09,10799.000000
75%,6.450000e+05,7.308900e+09,16198.000000
max,7.700000e+06,9.900000e+09,21597.000000
